In [2]:
from  scipy.integrate import quad
from scipy.special import gamma
import differint.differint as df
import matplotlib.pyplot as plt
import numpy as np

In [61]:
class DFSGOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, a=0, b=1, name="DFSGOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._is_first = True
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "pv") #previous variable i.e. weight or bias
        for var in var_list:
            self.add_slot(var, "pg") #previous gradient


    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
        # new_var_m = var - grad * lr_t
        print("DEBUG #1")
        new_var_m = var*2
        print("DEBUG #2")
        print(var.eval())
        print("DEBUG #2.1")
        roro = var[var<0]
        print(roro.shape)
        print("DEBUG #2.2")
        # new_var_m[var>0] = var[var>0] - lr_t * quad(lambda alpha: (2*(alpha-self.a))/(self.b-self.a)**2 * df.RL(alpha, f, np.zeros((var[var>0].shape[0], )), var[var>0])[-1], a=self.a, b=self.b)[0]
        new_var_m = var - grad * lr_t
        print("DEBUG #3")
        # new_var_m[var<0] = var[var<0] + lr_t * quad(lambda alpha: (2*(alpha-self.a))/(self.b-self.a)**2 * df.RL(alpha, f, var[var<0], np.zeros((var[var<0].shape[0], )))[-1], a=self.a, b=self.b)[0]
        print("DEBUG #4")

        pv_var = self.get_slot(var, "pv")
        pg_var = self.get_slot(var, "pg")
        
        if self._is_first:
            self._is_first = False
            new_var = new_var_m
        else:
            cond = grad*pg_var >= 0
            print(cond)
            avg_weights = (pv_var + var)/2.0
            new_var = tf.where(cond, new_var_m, avg_weights)
        pv_var.assign(var)
        pg_var.assign(grad)
        var.assign(new_var)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }


    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._serialize_hyperparameter("decay"),
            "momentum": self._serialize_hyperparameter("momentum"),
        }

In [62]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10, input_shape=[8]), tf.keras.layers.Dense(5), tf.keras.layers.Dense(1)])
model.compile(loss="mse", optimizer=DFSGOptimizer(learning_rate=0.001))
model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
DEBUG #1
DEBUG #2


AttributeError: in user code:

    c:\Users\Mohammad\.conda\envs\RL-TF\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\Users\Mohammad\AppData\Local\Temp\ipykernel_17992\1465144720.py:29 _resource_apply_dense  *
        print(var.eval())
    c:\Users\Mohammad\.conda\envs\RL-TF\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:624 eval  **
        return self._graph_element.eval(session=session)

    AttributeError: 'NoneType' object has no attribute 'eval'


In [ ]:
epoch = 5
for ep in range(epoch):
    for i in range(len(y_train)):
        w = model.get_weights()
        w = dist_frac_optimizer(w)
        model.set_weights(w)
        out = model.predict(X_train_scaled[i])
        w = dist_frac_optimizer(loss, w)

In [411]:
from tensorflow.keras.layers import Flatten
from  scipy.integrate import quad
import differint.differint as df
import itertools
import copy

class Net:
    def __init__(self, x, y, layers, batch_size=32):
        self.x = x
        self.y = y
        self.layers = layers
        self.w = self.init_weight(self.x, self.layers)
        self.w_shape = np.asarray([x.shape for x in self.w])
        self.w_flatten = self.flattener(self.w, self.w_shape)
        self.step=0
        self.batch_size = batch_size
        self.batch_counter = 0

    def init_weight(self, x, layers):
        w = []
        tmp = np.random.normal(0, 1, (x.shape[1], layers[0]))
        w.append(tmp / np.linalg.norm(tmp))
        for i in range(1, len(layers)):
            tmp = np.random.normal(0, 1, (layers[i-1], layers[i]))
            w.append(tmp / np.linalg.norm(tmp))

        return np.asarray(w)

    def flattener(self, w, w_shape):
        temp = [[xx for x in w[i] for xx in x] for i in range(len(w_shape))]
        flat_w = [xx for x in temp for xx in x]
        return np.asarray(flat_w)

    def forward(self, w):
        # print("forward calculate")
        H = self.x[self.batch_counter: self.batch_counter + self.batch_size] @ np.asarray(np.reshape(w[:self.w_shape[0][0] * self.w_shape[0][1]], self.w_shape[0]))
        for i in range(1, len(self.layers)):
            H = H @ np.asarray(np.reshape(w[self.w_shape[i-1][0] * self.w_shape[i-1][1]:(self.w_shape[i-1][0] * self.w_shape[i-1][1]) + (self.w_shape[i][0] * self.w_shape[i][1])], self.w_shape[i]))

        return H

    def forward_less(self):
        H = self.x[self.batch_counter: self.batch_counter + self.batch_size] @ np.asarray(np.reshape(self.w_flatten[:self.w_shape[0][0] * self.w_shape[0][1]], self.w_shape[0]))
        for i in range(1, len(self.layers)):
            H = H @ np.asarray(np.reshape(self.w_flatten[self.w_shape[i-1][0] * self.w_shape[i-1][1]:(self.w_shape[i-1][0] * self.w_shape[i-1][1]) + (self.w_shape[i][0] * self.w_shape[i][1])], self.w_shape[i]))

        return H

    def softmax(self, output):
        # print("softmax calculate")
        return (np.exp(output.T)/np.exp(output.T).sum(axis=0)).T
        # return np.asarray([np.asarray([np.exp(one)/ np.sum(np.exp(one_output)) for one in one_output]) for one_output in output])
    
    def MSE(self, w):
        return np.sum((self.y[self.batch_counter: self.batch_counter + self.batch_size] - self.softmax(self.forward())) ** 2)

    def categorical_cross_entropy(self, w):
        # we use this ([predict[i][np.argmax(target)]if predict[i][np.argmax(target)] != 0 else 10 ** 6]) for handling log(0) and preventing -Inf error.
        # print("loss calculate")
        # predict = self.softmax(self.forward(w))
        predict = self.softmax(self.forward_less())
        loss = -(1/self.batch_size) * np.sum([target * np.log([predict[i][np.argmax(target)]if predict[i][np.argmax(target)] != 0 else 0.1 ** 14]) for i, target in enumerate(self.y[self.batch_counter: self.batch_counter + self.batch_size])])
        # self.batch_counter += self.batch_size
        # print("batch_couter: ", self.batch_counter)
        return  loss

    def grad(self, f, x, epsilon=0.0001):
        # print("grad calculate")
        return (f(x + epsilon) - f(x)) / epsilon

    def optimizer(self, f, x0, lr=0.1, max_iter=100, return_history=False):
        x = x0
        history = [x]
        for i in range(max_iter):
            print(f"Iteration of Opt is: {i}, Loss is: {f(x)}")
            x_new = x - lr * self.grad(f, x)

            if f(x_new) < f(x):
                x = x_new
                self.w_flatten = copy.deepcopy(x_new)
            else:
                print(f"Updating learning rate: {lr}")
                lr = 0.1*lr
                if lr < 0.1 ** 10:
                    print("Early STOP!!!")
                    break
                
            history.append(x)
        if return_history:
            return x, history
        else:
            return x

    def frac_optimizer(self, f, x0, lr=0.1, alpha=0.9, max_iter=32, return_history=False):
        x = x0
        history = [x]
        x_new = copy.deepcopy(x)
        for i in range(max_iter):
            print(f"Iteration of Opt is: {i}, Loss is: {f(x)}")
            if (x>0).sum()>0:
                x_new[x>0] = x[x>0] - lr * df.RL(alpha, f, np.zeros((x.shape[0], )), x)[-1]
            if (x<0).sum()>0:                
                x_new[x<0] = x[x<0] + lr * df.RL(alpha, f, x, np.zeros((x.shape[0], )))[-1]

            if f(x_new) < f(x):
                x = x_new
                self.w_flatten = copy.deepcopy(x_new)
            else:
                print(f"Updating learning rate: {lr}")
                lr = 0.1*lr
                if lr < 0.1 ** 10:
                    print("Early STOP!!!")
                    break
            history.append(x)
            print(x)
        if return_history:
            return x, history
        else:
            return x


    def DFoptimizer(self, f,w0, lr=0.1, max_iter=100, a=0.3, b=0.95, return_history=False):
        print(f"Step is: {self.step}")
        self.step += 1
        x = w0
        x_new = copy.deepcopy(x)
        history = [x]
        for i in range(max_iter):
            print(f"Iteration of Opt is: {i}")
            x_new[x>0] = x[x>0] - lr * quad(lambda alpha: (2*(alpha-a))/(b-a)**2 * df.RL(alpha, f, np.zeros(x[x>0].shape, ), x[x>0])[-1], a=a, b=b)[0]
            x_new[x<0] = x[x<0] + lr * quad(lambda alpha: (2*(alpha-a))/(b-a)**2 * df.RL(alpha, f, x[x<0], np.zeros(x[x>0].shape, ))[-1], a=a, b=b)[0]
            if f(x_new) < f(x):
                x = x_new
            else:
                lr = 0.1*lr
            history.append(x)
        if return_history:
            return x, history
        else:
            return x

In [420]:
df.RL(0.95, model.categorical_cross_entropy, np.zeros((model.w_flatten.shape[0], )), model.w_flatten)

c:\Users\Mohammad\.conda\envs\RL-TF\lib\site-packages\differint\differint.py:324: RuntimeWarning: invalid value encountered in double_scalars
  RL = step_size**-alpha*np.dot(D, f_values)


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [422]:
len(model.w_flatten)

203264

In [412]:
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.python import metrics

def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = np.reshape(x_train, [-1, 28*28])
    x_test = np.reshape(x_test, [-1, 28*28])
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    x_train = x_train/np.max(x_train)
    x_test = x_test/np.max(x_test)
    return x_train, y_train, x_test, y_test

def run_tf_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(784,)))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    print("\nFitting Model...")
    model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2)
    print("\nEvaluating Model...")
    model.evaluate(x_test, y_test)


In [413]:
x_train, y_train, x_test, y_test= load_mnist()
model = Net(x_train, y_train, [256, 10], batch_size=8)
model.w_shape

C:\Users\Mohammad\AppData\Local\Temp\ipykernel_5268\4212714316.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(w)


array([[784, 256],
       [256,  10]])

In [414]:
output = model.forward(w=model.w_flatten)

In [415]:
out_soft = model.softmax(output)

In [416]:
model.categorical_cross_entropy(w=model.w_flatten)

2.3025912522358247

In [417]:
out_grad = model.grad(f=model.categorical_cross_entropy, x=model.w_flatten)

In [418]:
out_grad

2.552476874315701

In [ ]:
# grad optimizer
epoch = 5
for ep in range(epoch):
    print("EPOCH: ", ep)
    print(model.optimizer(model.categorical_cross_entropy, model.w_flatten))

In [419]:
# frac_optimizer
epoch = 5
for ep in range(epoch):
    print("EPOCH: ", ep)
    for b in range(int(x_test.shape[0]/model.batch_size)):
        print("Batch step: ", b)
        model.frac_optimizer(model.categorical_cross_entropy, model.w_flatten)
        model.batch_counter += model.batch_size

EPOCH:  0
Batch step:  0
Iteration of Opt is: 0, Loss is: 2.3025912522358247
Updating learning rate: 0.1
[-0.00193344 -0.00014224  0.00150529 ...  0.03034891  0.01840495
 -0.01418159]
Iteration of Opt is: 1, Loss is: 2.3025912522358247
Updating learning rate: 0.010000000000000002
[-0.00193344 -0.00014224  0.00150529 ...  0.03034891  0.01840495
 -0.01418159]
Iteration of Opt is: 2, Loss is: 2.3025912522358247
Updating learning rate: 0.0010000000000000002
[-0.00193344 -0.00014224  0.00150529 ...  0.03034891  0.01840495
 -0.01418159]
Iteration of Opt is: 3, Loss is: 2.3025912522358247
Updating learning rate: 0.00010000000000000003
[-0.00193344 -0.00014224  0.00150529 ...  0.03034891  0.01840495
 -0.01418159]
Iteration of Opt is: 4, Loss is: 2.3025912522358247
Updating learning rate: 1.0000000000000004e-05
[-0.00193344 -0.00014224  0.00150529 ...  0.03034891  0.01840495
 -0.01418159]
Iteration of Opt is: 5, Loss is: 2.3025912522358247
Updating learning rate: 1.0000000000000004e-06
[-0.0019

KeyboardInterrupt: 

In [ ]:
model.forward(model.w_flatten)

In [341]:
a=0.3
b=0.95
x=model.w_flatten
f=model.loss
quad(lambda alpha: (2*(alpha-a))/(b-a)**2 * df.RL(alpha, f, np.zeros(x[x>0].shape, ), x[x>0])[-1], a=a, b=b)[0], df.RL(0.9, f, np.zeros(x[x>0].shape, ), x[x>0])[-1], quad(lambda alpha: (2*(alpha-a))/(b-a)**2 * df.RL(alpha, f, x[x<0], np.zeros(x[x<0].shape, ))[-1], a=a, b=b)[0], df.RL(0.9, f, x[x<0], np.zeros(x[x<0].shape, ))[-1]

AttributeError: 'Net' object has no attribute 'loss'

In [66]:
model.DFoptimizer(f=model.loss, w0=model.w_flatten)

Step is: 0


ValueError: operands could not be broadcast together with shapes (19,) (17,) 

In [296]:
x_train, y_train, x_test, y_test= load_mnist()
run_tf_model(x_train, y_train, x_test, y_test)


Fitting Model...
Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6828 - accuracy: 0.8339 - val_loss: 0.3713 - val_accuracy: 0.9018
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3550 - accuracy: 0.9021 - val_loss: 0.3034 - val_accuracy: 0.9170
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3027 - accuracy: 0.9156 - val_loss: 0.2712 - val_accuracy: 0.9241
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2722 - accuracy: 0.9236 - val_loss: 0.2495 - val_accuracy: 0.9304
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2494 - accuracy: 0.9301 - val_loss: 0.2323 - val_accuracy: 0.9352

Evaluating Model...
313/313 [==============================] - 1s 2ms/step - loss: 0.2297 - accuracy: 0.9363


In [218]:
d = w.tolist()

In [219]:
d = [dd.tolist() for dd in d][0]

In [226]:
len(d)

14

In [387]:
model.w_flatten

array([0.98976572, 1.2954313 , 0.        , 0.04755698, 0.67182099,
       0.37223025, 0.        , 0.76523655, 0.        , 0.05103283,
       0.        , 0.72666734, 0.        , 0.        , 0.        ,
       0.86102577, 0.73552149, 0.        , 1.25006392, 0.        ,
       0.        , 0.        , 1.36286233, 2.13204624, 1.01869583,
       0.        , 0.        , 1.62782528, 0.19672655, 1.88353804,
       0.        , 0.34709588, 0.34611852, 0.42329038, 0.        ,
       0.7168049 ])

In [236]:
Flatten(w)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [120]:
len(model.w[0])

10

In [152]:
model.w_shape

array([[10,  5],
       [ 5,  1]])

In [151]:
model.w

array([array([[ 0.04933916,  0.31724101,  1.33483212, -0.36951808,  0.41092349],
              [ 0.05559069, -1.16103175, -0.28077493,  1.21152419,  1.14157878],
              [ 0.15441872, -0.3125879 ,  0.03497398,  0.1621402 , -1.90035389],
              [ 0.7068688 , -0.0317973 , -0.86584421,  0.32372658, -0.89127474],
              [-0.10104525,  0.96800556, -1.39178279,  2.27920435,  1.00664723],
              [ 0.82722813, -1.07102583,  0.35435052, -0.52659841, -1.37762987],
              [ 0.08271013, -0.96096173,  1.13707415, -0.67341641, -0.08163573],
              [ 0.17273798, -0.69070635,  1.37545584,  1.24066547,  0.70428081],
              [ 0.74952304, -1.11908976,  2.01680357, -1.51747064, -1.90236393],
              [-0.5939951 , -1.53041876,  0.97819404, -0.87736361, -1.27146795]]),
       array([[ 1.05409205],
              [ 0.14346276],
              [-0.24564847],
              [-0.06150027],
              [-0.5658025 ]])], dtype=object)

In [132]:
w_f = np.concatenate(model.w).ravel()

In [141]:
type(np.reshape(w_f[:50], model.w_shape[0])[0][0])

numpy.float64

In [136]:
np.reshape(w_f[:50], model.w_shape[0])

array([[-0.07864889,  2.42846981,  1.54135967, -0.66644431, -0.58885281],
       [-1.40438002,  0.40655187,  1.42136415, -2.25349914,  0.208802  ],
       [-0.85526952, -0.63766747,  0.28649079,  0.31262855, -0.3212233 ],
       [ 0.9956536 , -0.44415874, -0.26332207, -1.23753941, -0.55017982],
       [ 0.57276725, -2.44569546,  0.17469251, -0.06614036,  0.63622065],
       [ 0.67306601,  0.03192776, -0.03073269, -0.69936905,  0.91358389],
       [-0.46484078, -0.19981802, -0.62470099, -0.51558878, -0.94093217],
       [-0.70036259,  1.27562823,  0.16093373, -0.84742128, -0.33531485],
       [-0.01276554,  0.12688483,  0.19269448, -1.08818336, -0.17162264],
       [ 1.33220445, -1.06598381, -0.17105323,  1.73033611, -0.54751369]])

In [ ]:
epoch = 5
model = Net()
for ep in range(epoch):
    for i in range(len(y_train)):
        w = dist_frac_optimizer(w)
        model.set_weights(w)
        out = model.predict(X_train_scaled[i])
        w = dist_frac_optimizer(loss, w)

In [245]:
x = np.random.random((10,2))

In [247]:
v = np.array([3,2])

In [254]:
x.T

array([[0.2406876 , 0.18074978, 0.25898608, 0.43478112, 0.54350592,
        0.28659122, 0.20408715, 0.37472006, 0.89721325, 0.75278481],
       [0.11015054, 0.43815505, 0.17482847, 0.92732596, 0.14846509,
        0.24431521, 0.73363338, 0.1071827 , 0.32321506, 0.58601648]])

In [259]:
(np.exp(x.T)/np.exp(x.T).sum(axis=0)).T

array([[0.532588  , 0.467412  ],
       [0.43600166, 0.56399834],
       [0.52102699, 0.47897301],
       [0.37929425, 0.62070575],
       [0.59749558, 0.40250442],
       [0.51056743, 0.48943257],
       [0.37062273, 0.62937727],
       [0.56648823, 0.43351177],
       [0.63968522, 0.36031478],
       [0.54159572, 0.45840428]])

In [257]:
np.exp(x.T)

array([[1.27212356, 1.19811535, 1.29561577, 1.54462494, 1.7220336 ,
        1.33187965, 1.22640504, 1.45458416, 2.45275835, 2.12290367],
       [1.11644613, 1.54984518, 1.1910419 , 2.52774086, 1.1600523 ,
        1.27674671, 2.08263388, 1.11313761, 1.38156244, 1.79681648]])

In [258]:
np.exp(x.T).sum(axis=0)

array([2.38856969, 2.74796054, 2.48665767, 4.0723658 , 2.88208591,
       2.60862637, 3.30903891, 2.56772177, 3.83432079, 3.91972015])

In [313]:
modelK = Sequential()
modelK.add(Dense(256, activation='relu', input_shape=(784,)))
modelK.add(Dense(10, activation='softmax'))
modelK.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [314]:
modelK.evaluate(x_test,y_test)

313/313 [==============================] - 2s 3ms/step - loss: 2.3678 - accuracy: 0.1138


[2.3678197860717773, 0.11379999667406082]

In [315]:
modelK.fit(x_train, y_train, batch_size=60000, epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 490ms/step - loss: 2.3686 - accuracy: 0.1132
Epoch 2/5
1/1 [==============================] - 0s 92ms/step - loss: 2.3537 - accuracy: 0.1175
Epoch 3/5
1/1 [==============================] - 0s 96ms/step - loss: 2.3393 - accuracy: 0.1224
Epoch 4/5
1/1 [==============================] - 0s 88ms/step - loss: 2.3253 - accuracy: 0.1276
Epoch 5/5
1/1 [==============================] - 0s 91ms/step - loss: 2.3117 - accuracy: 0.1338


In [316]:
we = modelK.get_weights()

In [317]:
we

[array([[-0.01072868, -0.07011654, -0.04292168, ..., -0.06528095,
          0.07219957, -0.000182  ],
        [-0.004351  ,  0.03537294,  0.05221342, ..., -0.00466982,
          0.03416617, -0.00778186],
        [-0.03104708,  0.06621596,  0.01865447, ...,  0.04406609,
         -0.07491915, -0.00636803],
        ...,
        [-0.02563186,  0.00449546, -0.00927953, ..., -0.03727087,
          0.05514884,  0.05221426],
        [-0.01084206, -0.02047814, -0.03697304, ..., -0.02528916,
          0.02550144,  0.07291465],
        [-0.02149177,  0.07128689,  0.00808644, ...,  0.03317444,
         -0.00483374,  0.0458691 ]], dtype=float32),
 array([-1.84047298e-04,  6.38793208e-05,  2.76196486e-04,  2.35976418e-04,
         1.66774364e-04, -1.85569425e-05, -3.79865320e-04,  3.47005553e-05,
         2.44475639e-04,  5.67853874e-07, -2.41147543e-04, -1.89210012e-04,
        -1.83290576e-05, -4.82491712e-04, -4.00788886e-05,  8.42920781e-05,
         9.57026787e-05, -5.61380526e-04,  6.39556019e

In [318]:
model.w_flatten

array([ 1.11834062,  0.47999893, -1.56807566, ...,  0.29614132,
       -0.53334974,  0.69503652])